In [1]:
import pyspark
from pyspark.sql import SparkSession, types
from pyspark.sql import functions as F

In [2]:
#!wget https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2023-01.parquet

/Users/dmytriyt/Desktop: Scheme missing.
--2024-01-04 11:14:42--  https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2023-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.244.96.180, 18.244.96.103, 18.244.96.218, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.244.96.180|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 473816636 (452M) [application/x-www-form-urlencoded]
Saving to: 'fhvhv_tripdata_2023-01.parquet'

fhvhv_tripdata_2023 100%[===================>] 451.87M  50.6MB/s    in 8.7s    

2024-01-04 11:14:51 (51.7 MB/s) - 'fhvhv_tripdata_2023-01.parquet' saved [473816636/473816636]

FINISHED --2024-01-04 11:14:51--
Total wall clock time: 8.9s
Downloaded: 1 files, 452M in 8.7s (51.7 MB/s)


In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/04 11:15:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/01/04 11:15:43 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [31]:
df = spark.read \
    .parquet('fhvhv_tripdata_2023-01.parquet')

In [32]:
license_names = {
    'HV0002': 'Juno',
    'HV0003': 'Uber',
    'HV0004': 'Via',
    'HV0005': 'Lyft',
}

def license_name(num_string):
    if num_string in license_names:
        return license_names[num_string]
    else:
        return 'Unknown'

license_name_udf = F.udf(license_name, returnType=types.StringType())

In [33]:
df_clean = df \
    .withColumn('license_name', license_name_udf(df.hvfhs_license_num)) \
    .select(
        'pickup_datetime', 'hvfhs_license_num', 'license_name',
        'trip_miles', 'trip_time', 'base_passenger_fare', 'tolls',
        'bcf', 'sales_tax', 'congestion_surcharge', 'airport_fee', 'tips', 'driver_pay'
    )

In [34]:
df_clean.repartition('hvfhs_license_num')

DataFrame[pickup_datetime: timestamp_ntz, hvfhs_license_num: string, license_name: string, trip_miles: double, trip_time: bigint, base_passenger_fare: double, tolls: double, bcf: double, sales_tax: double, congestion_surcharge: double, airport_fee: double, tips: double, driver_pay: double]

In [35]:
df_clean.registerTempTable('fhvhv_tripdata')

/usr/local/Cellar/apache-spark/3.5.0/libexec/python/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [36]:
report = spark.sql("""
SELECT
    date_trunc('day', pickup_datetime) trip_day
    ,license_name
    ,sum(trip_miles) total_miles
    ,avg(trip_miles) avg_miles_a_day
    ,sum(base_passenger_fare) total_base_fare
    ,sum(tolls) total_tolls
    ,sum(bcf) total_bcf
    ,sum(congestion_surcharge) total_congestion_surcharge
    ,sum(airport_fee) airport_fee
    ,sum(tips) total_tips
FROM fhvhv_tripdata 
GROUP BY 1,2
ORDER BY 1,2
LIMIT 100
""")

In [37]:
report.write \
    .mode("overwrite") \
    .csv('report/', )